In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install lifelines -q

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 349.3/349.3 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.3/117.3 kB 5.9 MB/s eta 0:00:00


In [5]:
TRAIN_PATH = "/content/drive/MyDrive/1:1_Shriya_Wagholikar/Datasets/experiment_2a/exp2A_selectKBest_train.csv"
TEST_PATH = "/content/drive/MyDrive/1:1_Shriya_Wagholikar/Datasets/experiment_2a/exp2A_selectKBest_test.csv"

In [6]:
import pandas as pd
train_data = pd.read_csv(TRAIN_PATH)
train_data.head()

,ASB13,BEX4,BRCA1,C6orf58,TMEM248,CALU,CDK2,DNAJB12,EN1,FAM114A1,...,TEF,TGIF1,TMEM194B,TNFRSF11B,TNFRSF12A,TTC26,TUBGCP2,WISP1,Disease Free (Months),DFS_STATUS_ENCODED
0,-1.8265,0.4752,-0.6760,-0.2004,2.3701,-0.0445,-0.7401,-0.6147,1.3637,0.6627,...,-0.0560,1.6604,0.5439,1.6735,0.7437,1.9809,-0.8906,-0.9555,2.43,0.0
1,-0.4475,1.4117,-0.4643,-0.2125,-1.4078,-0.8186,0.2352,-0.0811,-0.0143,-1.6606,...,-0.2006,-0.7564,-0.1001,-0.0141,-0.1963,-1.0834,0.7203,-1.2828,20.20,1.0
2,-0.3157,1.2607,-1.4290,-0.3844,-0.5240,-0.9108,-1.6550,0.1379,-0.0853,0.1569,...,1.6264,-1.4512,-0.3695,-0.0955,-0.8809,-1.1376,0.2028,0.4800,20.43,0.0
3,0.3588,0.4288,-0.3246,-0.9091,-1.0282,-0.4820,-0.8674,0.5796,-0.4008,-0.2822,...,0.7224,-1.0325,-0.4064,-1.2270,-1.2200,-0.5648,0.5372,-0.7155,8.18,0.0
4,1.2976,0.2353,-1.0768,-1.9452,-0.5185,-0.1219,-0.6325,0.5071,-1.0241,-1.2173,...,1.1507,-0.4716,-1.5549,-0.6189,0.2553,-1.5268,1.0314,-0.4206,31.80,0.0


In [7]:
test_data = pd.read_csv(TEST_PATH)
test_data.head()

,ASB13,BEX4,BRCA1,C6orf58,TMEM248,CALU,CDK2,DNAJB12,EN1,FAM114A1,...,TEF,TGIF1,TMEM194B,TNFRSF11B,TNFRSF12A,TTC26,TUBGCP2,WISP1,Disease Free (Months),DFS_STATUS_ENCODED
0,-0.1050,1.0051,-1.2564,2.1691,-0.6761,-0.2567,-1.2467,0.1240,-0.9673,-0.9148,...,0.6173,-0.6709,-1.0707,-0.5709,-0.3455,-0.1503,1.3146,-0.2024,119.48,1.0
1,-0.5950,0.5236,-0.4643,-0.2513,-0.2209,-1.0171,0.0826,1.2554,-0.9815,0.2250,...,-0.1429,0.9833,0.6752,0.0074,0.8362,0.0404,0.8986,-0.3668,95.86,0.0
2,0.2187,-0.6935,0.5225,-1.9452,-0.3263,0.1202,0.3435,0.3733,0.2466,0.3526,...,-1.4228,0.0080,-0.4799,1.5829,-0.3940,0.6889,0.3286,-0.5494,16.79,1.0
3,0.5543,0.5403,-0.2203,1.1511,-0.6104,0.3412,-0.2451,-0.1870,-0.7498,-0.0701,...,0.9072,-0.3368,-0.8850,-0.2086,-0.2341,-0.0366,0.3464,-0.2734,14.55,0.0
4,0.3029,-0.1179,-1.1101,0.6479,-0.7701,-0.1898,-0.4056,0.5613,-0.5275,-0.1509,...,0.7651,-0.6792,-0.3531,-0.7662,0.1757,-0.5349,0.7937,0.8867,31.18,0.0


In [ ]:
# #in the event you have to drop columns -> only execute if you have some columns to drop in step 2, which we don't have currently
# cols_to_drop = []
# train_data.drop(cols_to_drop, axis=1, inplace=True)
# test_data.drop(cols_to_drop, axis=1, inplace=True)


In [8]:
from sklearn.model_selection import train_test_split, KFold
from lifelines import CoxPHFitter
from lifelines.utils import concordance_index
import numpy as np
def apply_cross_validation(df:pd.DataFrame, penalty: float = 0.1) -> None:
    # create the folds
    kf = KFold(n_splits=5, shuffle=True, random_state=42)
    c_index_scores = []
    # run cross validation
    for train_index, val_index in kf.split(train_data):
        train_fold = df.iloc[train_index]
        val_fold = df.iloc[val_index]
        cph = CoxPHFitter(penalizer = penalty)
        cph.fit(train_fold, duration_col='Disease Free (Months)', event_col='DFS_STATUS_ENCODED')
        val_predictions = cph.predict_partial_hazard(val_fold)
        c_index = concordance_index(val_fold['Disease Free (Months)'], -val_predictions, val_fold['DFS_STATUS_ENCODED'])
        c_index_scores.append(c_index)
    print(f"Average Validation C-index: {np.mean(c_index_scores)}")

In [19]:
apply_cross_validation(train_data, penalty = 1)

Average Validation C-index: 0.7354255873884987


In [20]:
# Train the final model on the entire training set
cph_final = CoxPHFitter(penalizer = 0.9)
# fit on train data
cph_final.fit(train_data, duration_col='Disease Free (Months)', event_col='DFS_STATUS_ENCODED')
cph_final.print_summary()

<lifelines.CoxPHFitter: fitted with 382 total observations, 250 right-censored observations>
             duration col = 'Disease Free (Months)'
                event col = 'DFS_STATUS_ENCODED'
                penalizer = 0.9
                 l1 ratio = 0.0
      baseline estimation = breslow
   number of observations = 382
number of events observed = 132
   partial log-likelihood = -603.53
         time fit was run = 2025-10-16 23:54:01 UTC

---
           coef exp(coef)  se(coef)  coef lower 95%  coef upper 95% exp(coef) lower 95% exp(coef) upper 95%
covariate                                                                                                  
ASB13     -0.04      0.96      0.05           -0.14            0.06                0.87                1.06
BEX4      -0.01      0.99      0.05           -0.11            0.10                0.89                1.10
BRCA1      0.04      1.04      0.05           -0.06            0.14                0.94                1.15
C6orf58   -0.05      0.95      0.04           -0.14            0.03                0.87                1.03
TMEM248    0.00      1.00      0.05           -0.10            0.10                0.91                1.10
CALU       0.01      1.01      0.05           -0.10            0.11                0.91                1.12
CDK2       0.01      1.01      0.05           -0.08            0.11                0.92                1.12
DNAJB12   -0.03      0.97      0.05           -0.13            0.07                0.88                1.07
EN1        0.05      1.06      0.05           -0.04            0.15                0.96                1.17
FAM114A1  -0.00      1.00      0.05           -0.11            0.10                0.90                1.11
FAM155A   -0.04      0.96      0.05           -0.13            0.06                0.88                1.06
FAM46A     0.04      1.04      0.05           -0.06            0.14                0.94                1.15
FNDC3B     0.01      1.01      0.05           -0.10            0.11                0.91                1.12
GALNT2     0.00      1.00      0.05           -0.10            0.10                0.91                1.11
GGCX       0.03      1.03      0.05           -0.07            0.13                0.93                1.14
GNG12      0.01      1.01      0.05           -0.09            0.11                0.91                1.12
DNAAF5     0.02      1.02      0.05           -0.08            0.12                0.92                1.13
HOXC4      0.04      1.04      0.05           -0.05            0.14                0.95                1.15
HOXC5      0.02      1.02      0.05           -0.08            0.12                0.92                1.13
HOXC6      0.04      1.04      0.06           -0.07            0.15                0.93                1.16
HS2ST1     0.00      1.00      0.05           -0.10            0.10                0.91                1.11
IDI1      -0.00      1.00      0.05           -0.10            0.10                0.91                1.10
CAMSAP3   -0.02      0.98      0.05           -0.12            0.08                0.89                1.08
LOC400043  0.03      1.03      0.05           -0.07            0.13                0.94                1.13
LRIT2     -0.01      0.99      0.05           -0.11            0.10                0.90                1.10
LRRC20    -0.01      0.99      0.05           -0.11            0.10                0.90                1.10
MAGEH1    -0.03      0.97      0.05           -0.13            0.07                0.88                1.08
MRFAP1L1  -0.06      0.94      0.05           -0.15            0.04                0.86                1.04
MSN        0.01      1.01      0.05           -0.10            0.11                0.91                1.12
NGFRAP1   -0.00      1.00      0.05           -0.10            0.10                0.91                1.10
PSTK      -0.04      0.96      0.05           -0.14            0.06                0.87      

In [21]:
# Concordance Index
c_index = cph_final.concordance_index_
print(f"Concordance Index: {c_index}")

Concordance Index: 0.7580365817598107


In [22]:
import pickle
MODEL_PATH = "/content/drive/MyDrive/1:1_Shriya_Wagholikar/Results/experiment_2a/best_model_exp2a"
with open(MODEL_PATH, 'wb') as file:
    pickle.dump(cph_final, file)

In [24]:
# define the train and test path
UPDATED_TRAIN_PATH = "/content/drive/MyDrive/1:1_Shriya_Wagholikar/Datasets/experiment_2a/updated_exp2a_selectKBest_train.csv"
UPDATED_TEST_PATH = "/content/drive/MyDrive/1:1_Shriya_Wagholikar/Datasets/experiment_2a/updated_exp2a_selectKBest_test.csv"
train_data.to_csv(UPDATED_TRAIN_PATH, index=False)
test_data.to_csv(UPDATED_TEST_PATH, index=False)